# copied over from last year DL course, adapt to this year's assgn

## Q3


In [ ]:
import tqdm as notebook_tqdm
import ipywidgets as widgets
from datasets import load_dataset
dataset = load_daaset("wmt16",'de-en')

training_words = dataset['train']
print(type(training_words))
training_words=training_words.select(range(100000))

eval_words = dataset['validation']
test_words = dataset['test']

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

BadZipFile: File is not a zip file

In [ ]:
from transformers import AutoTokenizer

checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

C:\Users\Gautam\AppData\Roaming\Python\Python310\site-packages\transformers\models\t5\tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
source_lang = "en"
target_lang = "de"
prefix = "translate English to German: "

def preprocess_function(examples):
    inputs = [prefix + example[source_lang] for example in examples["translation"]]
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

In [ ]:
tokenized__train = training_words.map(preprocess_function, batched=True)
tokenized_eval= eval_words.map(preprocess_function, batched=True)
tokenized_test= test_words.map(preprocess_function, batched=True)

Loading cached processed dataset at C:\Users\Gautam\.cache\huggingface\datasets\wmt16\de-en\1.0.0\746749a11d25c02058042da7502d973ff410e73457f3d305fc1177dc0e8c4227\cache-0b400da7fc23c8aa.arrow
Loading cached processed dataset at C:\Users\Gautam\.cache\huggingface\datasets\wmt16\de-en\1.0.0\746749a11d25c02058042da7502d973ff410e73457f3d305fc1177dc0e8c4227\cache-ec2f9a3876d81dd4.arrow
Loading cached processed dataset at C:\Users\Gautam\.cache\huggingface\datasets\wmt16\de-en\1.0.0\746749a11d25c02058042da7502d973ff410e73457f3d305fc1177dc0e8c4227\cache-23e41396a5aab6dc.arrow


In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [ ]:
import evaluate

blue = evaluate.load("bleu")
rouge= evaluate.load("rouge")
import numpy as np


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
                          
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result_bleu = blue.compute(predictions=decoded_preds, references=decoded_labels)
    # result_bleu2 = bleu2.compute(predictions=decoded_preds, references=decoded_labels)[0]
    rougeL = rouge.compute(predictions=decoded_preds, references=decoded_labels)

    result = {"bleu1": result_bleu['precisions'][0],"bleu2": result_bleu['precisions'][1],"rougeL":rougeL['rougeL']}

    # prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    # result["gen_len"] = np.mean(prediction_lens)
    # result = {k: round(v, 4) for k, v in result.items()}
    return result





# def compute_metrics(eval_preds):
#     preds, labels = eval_preds
#     if isinstance(preds, tuple):
#         preds = preds[0]
#     decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
                          
#     labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
#     decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

#     decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

#     result_bleu = blue.compute(predictions=decoded_preds, references=decoded_labels)
#     result_rougeL = rouge.compute(predictions=decoded_preds, references=decoded_labels)

#     result = {"bleu1": result_bleu,'rouge_l':result_rougeL}

    # prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    # result["gen_len"] = np.mean(prediction_lens)
    # result = {k: round(v, 4) for k, v in result.items()}


In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
# print(model)

for i,param in enumerate(model.parameters()):
    if i <35:
        param.requires_grad=False
    print(i,param.requires_grad)

0 False
1 False
2 False
3 False
4 False
5 False
6 False
7 False
8 False
9 False
10 False
11 False
12 False
13 False
14 False
15 False
16 False
17 False
18 False
19 False
20 False
21 False
22 False
23 False
24 False
25 False
26 False
27 False
28 False
29 False
30 False
31 False
32 False
33 False
34 False
35 True
36 True
37 True
38 True
39 True
40 True
41 True
42 True
43 True
44 True
45 True
46 True
47 True
48 True
49 True
50 True
51 True
52 True
53 True
54 True
55 True
56 True
57 True
58 True
59 True
60 True
61 True
62 True
63 True
64 True
65 True
66 True
67 True
68 True
69 True
70 True
71 True
72 True
73 True
74 True
75 True
76 True
77 True
78 True
79 True
80 True
81 True
82 True
83 True
84 True
85 True
86 True
87 True
88 True
89 True
90 True
91 True
92 True
93 True
94 True
95 True
96 True
97 True
98 True
99 True
100 True
101 True
102 True
103 True
104 True
105 True
106 True
107 True
108 True
109 True
110 True
111 True
112 True
113 True
114 True
115 True
116 True
117 True
118 True
119 

In [ ]:
print( model.encoder.block[0].layer[1])

T5LayerFF(
  (DenseReluDense): T5DenseActDense(
    (wi): Linear(in_features=512, out_features=2048, bias=False)
    (wo): Linear(in_features=2048, out_features=512, bias=False)
    (dropout): Dropout(p=0.1, inplace=False)
    (act): ReLU()
  )
  (layer_norm): T5LayerNorm()
  (dropout): Dropout(p=0.1, inplace=False)
)


In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="translation",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
    logging_strategy='epoch'
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized__train,
    eval_dataset=tokenized_eval,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)




trainer.train()

In [ ]:
trainer.save_model("translationV2")

In [ ]:
loaded_model = AutoModelForSeq2SeqLM.from_pretrained('translation\checkpoint-25000')

In [ ]:
test_trainer = Seq2SeqTrainer(
    model=loaded_model,
    args=training_args,
    train_dataset=tokenized__train,
    eval_dataset=tokenized_eval,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


In [ ]:
trainer.predict(tokenized_test)

  0%|          | 0/188 [00:00<?, ?it/s]

PredictionOutput(predictions=array([[    0,  4534,     3, ...,     0,     0,     0],
       [    0,   316, 21583, ...,     1,     0,     0],
       [    0,   316,  5877, ...,    67,    20, 19614],
       ...,
       [    0,   316,  2415, ...,    64,  1008,   402],
       [    0,  1318, 13942, ...,  5007,     7,   173],
       [    0,  3139,  9237, ...,  1199,    10,  2442]], dtype=int64), label_ids=array([[ 4534,     3,    15, ...,  -100,  -100,  -100],
       [  644,     3, 20909, ...,  -100,  -100,  -100],
       [  316,  5877,    29, ...,  -100,  -100,  -100],
       ...,
       [  316,  2974,  1847, ...,  -100,  -100,  -100],
       [ 1318, 13942,    35, ...,  -100,  -100,  -100],
       [ 3139,  9237,     3, ...,  -100,  -100,  -100]], dtype=int64), metrics={'test_loss': 1.1579248905181885, 'test_bleu1': 0.6142907406321468, 'test_bleu2': 0.36281222875880026, 'test_rougeL': 0.4404123001219971, 'test_runtime': 47.6737, 'test_samples_per_second': 62.907, 'test_steps_per_second': 3.94